# Time Series Simulation Power Net

We start with the time series simulation of a power net:

In [ ]:
import numpy as np
import pandas as pd

data_pv = pd.read_excel(r'net_data_power/time_series.xlsx', sheet_name='pv_solar', index_col = 'time')
data_ho = pd.read_excel(r'net_data_power/time_series.xlsx', sheet_name='households', index_col = 'time')

In [ ]:
data_pv

In [ ]:
data_ho

In [ ]:
data_ho = data_ho.loc[data_pv.index]

In [ ]:
import pandapower as pp
net = pp.from_json(r'pp/net.json')

In [ ]:
profs_name_ho = np.random.choice(['A', 'B', 'C'], len(net.load.index))
profs_name_pv = np.random.choice(['pv_south_nom', 'pv_southwest_nom'], len(net.sgen.index))

In [ ]:
profs_name_ho

In [ ]:
profs_name_pv

In [ ]:
profs_name_ho_p = ['H0-%s_pload' %ho for ho in profs_name_ho]
profs_name_ho_q = ['H0-%s_qload' %ho for ho in profs_name_ho] 

In [ ]:
datasource_ho_p = data_ho.loc[:, data_ho.columns.str.contains('pload')] * 0.06
datasource_ho_q = data_ho.loc[:, data_ho.columns.str.contains('qload')] * 0.02
datasource_pv = data_pv * 0.03

In [ ]:
from pandapower.control import ConstControl
from pandapower.timeseries import DFData

ConstControl(net, 'load', 'p_mw', net.load.index, profs_name_ho_p, DFData(datasource_ho_p))
ConstControl(net, 'load', 'q_mvar', net.load.index, profs_name_ho_q, DFData(datasource_ho_q))
ConstControl(net, 'sgen', 'p_mw', net.sgen.index, profs_name_pv, DFData(datasource_pv))

In [ ]:
net.controller

In [ ]:
from pandapower.timeseries import OutputWriter
log_variables = [('res_bus', 'vm_pu'), ('res_line', 'loading_percent'), ('res_trafo', 'loading_percent')]
OutputWriter(net, data_pv.index, 'net_data_power/results', log_variables=log_variables)

In [ ]:
from pandapower.timeseries.run_time_series import run_timeseries
run_timeseries(net, data_pv.index)

# Time Series Simulation Pipe Net

In [ ]:
data_el = pd.DataFrame(np.random.random(168), columns=['electrolyzer'], index=data_pv.index) * 0.001

In [ ]:
import pandapipes as pps
mn = pps.from_json(r'multinet.json')

In [ ]:
gnet = mn.nets['gas']

In [ ]:
gnet.source

In [ ]:
ConstControl(gnet, 'source', 'mdot_kg_per_s', 3, ['electrolyzer'], DFData(data_el))

In [ ]:
log_variables = [('res_pipe', 'v_mean_m_per_s'), ('res_junction', 'p_bar'), ('res_junction', 't_k')]
OutputWriter(gnet, data_pv.index, 'net_data_pipe/results', log_variables=log_variables)

In [ ]:
from pandapipes.timeseries.run_time_series import run_timeseries

In [ ]:
run_timeseries(gnet, data_pv.index, iter=20)

# Time Series Simulation Multinet

In [ ]:
data_p2g = pd.DataFrame(np.random.random(168) * 0.2, columns=['P2G'], index=data_pv.index)
mn = pps.from_json(r'multinet.json')
enet = mn['nets']['power']
gnet = mn['nets']['gas']
enet.load

In [ ]:
ConstControl(enet, 'load', 'p_mw', 8, ['P2G'], DFData(data_p2g))

In [ ]:
log_variables = [('res_pipe', 'v_mean_m_per_s'), ('res_junction', 'p_bar'), ('res_junction', 't_k')]
OutputWriter(gnet, data_pv.index, 'results', log_variables=log_variables)
log_variables = [('res_bus', 'vm_pu'), ('res_line', 'loading_percent'), ('res_trafo', 'loading_percent')]
OutputWriter(enet, data_pv.index, 'results', log_variables=log_variables)

In [ ]:
from pandapipes.multinet.timeseries.run_time_series_multinet import run_timeseries
run_timeseries(mn, data_pv.index, iter=20)